In [1]:
import boto3
import sagemaker
import pandas as pd
import numpy as np

from sagemaker.predictor import Predictor
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from tqdm import tqdm

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
X_test = pd.read_csv("../data/processed/X_test.csv")
y_test = pd.read_csv("../data/processed/y_test.csv")

# Si y_test viene como dataframe de una columna
if isinstance(y_test, pd.DataFrame):
    y_test = y_test.iloc[:, 0]

print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_test shape: (749, 20)
y_test shape: (749,)


In [3]:
def reconstruct_original_features(X):

    X_reconstructed = X.copy()

    # -------- how_you_were_involved --------
    involved_cols = [col for col in X.columns if col.startswith("how_you_were_involved_")]

    def get_involved(row):
        for col in involved_cols:
            if row[col] == 1:
                value = col.replace("how_you_were_involved_", "")
                
                if value == "Not_involved":
                    return np.nan
                
                return value
        return np.nan

    X_reconstructed["how_you_were_involved"] = X.apply(get_involved, axis=1)

    # -------- state_accident_occur --------
    state_cols = [col for col in X.columns if col.startswith("state_accident_occur_")]

    def get_state(row):
        for col in state_cols:
            if row[col] == 1:
                return col.replace("state_accident_occur_", "")
        return np.nan

    X_reconstructed["state_accident_occur"] = X.apply(get_state, axis=1)

    # Eliminar columnas one-hot
    X_reconstructed = X_reconstructed.drop(columns=involved_cols + state_cols)

    # Eliminar target si viene incluido
    if "lawyer_status" in X_reconstructed.columns:
        X_reconstructed = X_reconstructed.drop(columns=["lawyer_status"])

    return X_reconstructed

In [4]:
X_test_original = reconstruct_original_features(X_test)

print("Columnas reconstruidas:")
print(X_test_original.columns)
print("Shape:", X_test_original.shape)

Columnas reconstruidas:
Index(['were_the_police_call', 'did_the_ambulance_arrive_at_the_scene',
       'did_the_ambulance_take_you_toahospital',
       'did_the_police_arrived_at_the_scene', 'did_the_police_fileareport',
       'did_you_get_injured_byaslip_or_fall_accident',
       'did_you_have_an_accident_at_work', 'days_since_accident',
       'were_you_affected_by_possible_malpractice',
       'were_you_involved_in_an_automobile_accident', 'how_you_were_involved',
       'state_accident_occur'],
      dtype='object')
Shape: (749, 12)


In [5]:
region = "us-east-1"
endpoint_name = "xgboost-excalibur"

sagemaker_session = sagemaker.Session()
sagemaker_client = boto3.client("sagemaker", region_name=region)

predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)

print("Conectado al endpoint:", endpoint_name)

Conectado al endpoint: xgboost-excalibur


In [6]:
def row_to_payload(row):
    payload = row.to_dict()
    
    for k, v in payload.items():
        if pd.isna(v):
            payload[k] = None
    
    return payload

In [7]:
predictions = []

for _, row in tqdm(X_test_original.iterrows(), total=len(X_test_original)):
    
    payload = row_to_payload(row)
    response = predictor.predict(payload)
    
    # Ajustar según formato de respuesta
    if isinstance(response, dict):
        prob = list(response.values())[0]
    else:
        prob = response
    
    predictions.append(prob)

  0%|          | 0/749 [00:00<?, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:6                                                                                    │
│                                                                                                  │
│    3 for _, row in tqdm(X_test_original.iterrows(), total=len(X_test_original)):                 │
│    4 │                                                                                           │
│    5 │   payload = row_to_payload(row)                                                           │
│ ❱  6 │   response = predictor.predict(payload)                                                   │
│    7 │                                                                                           │
│    8 │   # Ajustar según formato de respuesta                                                    │
│    9 │   if isinstance(response, dict):                                                          │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/base_predictor.py:212 in predict               │
│                                                                                                  │
│   209 │   │   if inference_component_name:                                                       │
│   210 │   │   │   request_args["InferenceComponentName"] = inference_component_name              │
│   211 │   │                                                                                      │
│ ❱ 212 │   │   response = self.sagemaker_session.sagemaker_runtime_client.invoke_endpoint(**req   │
│   213 │   │   return self._handle_response(response)                                             │
│   214 │                                                                                          │
│   215 │   def _handle_response(self, response):                                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:569 in _api_call                      │
│                                                                                                  │
│    566 │   │   │   │   │   f"{py_operation_name}() only accepts keyword arguments."              │
│    567 │   │   │   │   )                                                                         │
│    568 │   │   │   # The "self" in this scope is referring to the BaseClient.                    │
│ ❱  569 │   │   │   return self._make_api_call(operation_name, kwargs)                            │
│    570 │   │                                                                                     │
│    571 │   │   _api_call.__name__ = str(py_operation_name)                                       │
│    572                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/botocore/client.py:1023 in _make_api_call                │
│                                                                                                  │
│   1020 │   │   │   │   "Code"                                                                    │
│   1021 │   │   │   )                                                                             │
│   1022 │   │   │   error_class = self.exceptions.from_code(error_code)                           │
│ ❱ 1023 │   │   │   raise error_class(parsed_response, operation_name)                            │
│   1024 │   │   else:                                                                             │
│   1025 │   │   │   return parsed_response                                                        │
│   1026                                                     

In [ ]:
predictions = np.array(predictions)
predicted_labels = (predictions >= 0.5).astype(int)

In [ ]:
accuracy = accuracy_score(y_test, predicted_labels)
precision = precision_score(y_test, predicted_labels)
recall = recall_score(y_test, predicted_labels)
f1 = f1_score(y_test, predicted_labels)

print("📊 Métricas del modelo en el endpoint")
print("--------------------------------------")
print("Accuracy :", accuracy)
print("Precision:", precision)
print("Recall   :", recall)
print("F1 Score :", f1)

print("\nClassification Report:\n")
print(classification_report(y_test, predicted_labels))